In [1]:
!kinit -kt $KRB5_CLIENT_KTNAME $KRB5_PRINCIPAL

In [1]:
%%configure -f 
{
    "conf":
    {
        "spark.app.name":"Bhavesh_Demand_Forecast_Notebook",
        "spark.yarn.queue": "default",
        "spark.jars": "/apps/Jars/obelisk-retail-legos-Bhavesh.jar,/apps/Jars/mysql-connector-java-5.1.40.jar",
        "spark.driver.extraJavaOptions": "-Dscala.color",
        "spark.driver.memory": "2g",
        "spark.executor.memory": "2g",
        "spark.executor.instances": "4"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1512,application_1686980170466_0869,spark,idle,Link,Link,
1515,application_1686980170466_0883,spark,idle,Link,Link,


In [2]:
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import ai.couture.obelisk.commons.utils.DataFrameUtil.minMaxScaler
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1517,application_1686980170466_0885,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import ai.couture.obelisk.commons.utils.DataFrameUtil.minMaxScaler
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._


In [3]:
var productAttrs = {
    ParquetToDF.getDF("/data/ecomm/ajio/processed/productAttributesLegosFNL")
    .select(col("itemid") as "productid", col("colorfamily"), col("pattern"), col("price"), col("sleeve"), col("styletype"))
}

var sleeveNormalised = {
    CSVToDF.getDF("/data/ecomm/ajio/processed/inventory/attributes/Sleevelength_normalised.csv")
      .withColumnRenamed("attribute", "sleeve")
      .withColumnRenamed("synonym", "normalised_sleeve")
}

var styleTypeNormalised = {
    CSVToDF.getDF("/data/ecomm/ajio/processed/inventory/attributes/Styletype_normalised.csv")
      .withColumnRenamed("attribute", "styletype")
      .withColumnRenamed("synonym", "normalised_styletype")
}

var patternNormalised = {
    CSVToDF.getDF("/data/ecomm/ajio/processed/inventory/attributes/Pattern_normalised.csv")
      .withColumnRenamed("attribute", "pattern")
      .withColumnRenamed("synonym", "normalised_pattern")
}

productAttrs = {
    productAttrs.na.fill("")
      .join(sleeveNormalised.na.fill(""), Seq("sleeve"), "inner")
      .join(styleTypeNormalised.na.fill(""), Seq("styletype"), "inner")
      .join(patternNormalised.na.fill(""), Seq("pattern"), "inner")
      .drop("pattern", "sleeve", "styletype")
      .withColumnRenamed("normalised_sleeve", "sleeve")
      .withColumnRenamed("normalised_styletype", "styletype")
      .withColumnRenamed("normalised_pattern", "pattern")
}

val cols = productAttrs.columns
productAttrs = cols.foldLeft(productAttrs) { (memoDf, attr) =>
  memoDf.withColumn(attr, when(col(attr) === "", null).otherwise(col(attr)))
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productAttrs: org.apache.spark.sql.DataFrame = [productid: string, colorfamily: string ... 4 more fields]
sleeveNormalised: org.apache.spark.sql.DataFrame = [sleeve: string, normalised_sleeve: string]
styleTypeNormalised: org.apache.spark.sql.DataFrame = [styletype: string, normalised_styletype: string]
patternNormalised: org.apache.spark.sql.DataFrame = [pattern: string, normalised_pattern: string]
productAttrs: org.apache.spark.sql.DataFrame = [productid: string, colorfamily: string ... 4 more fields]
cols: Array[String] = Array(productid, colorfamily, price, sleeve, styletype, pattern)
productAttrs: org.apache.spark.sql.DataFrame = [productid: string, colorfamily: string ... 4 more fields]


In [4]:
var queryAvg = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/NMonthsAheadExperiment/combinedProductDerivedFromQueryPredictions")
queryAvg = {
    queryAvg.filter(col("viewsFraction")>0 && col("quantityFraction")>0)
    .groupBy("query", "similargrouplevel")
    .agg(avg("m1") as "m1", avg("m2") as "m2", avg("m3") as "m3", avg("m4") as "m4", avg("m5") as "m5", avg("m6") as "m6")
}
queryAvg.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryAvg: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: int ... 9 more fields]
queryAvg: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: int ... 6 more fields]
root
 |-- query: string (nullable = true)
 |-- similargrouplevel: integer (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)



### Gant

In [3]:
var gant = CSVToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/M&S Gant Aarke - Gant.csv", inferSchema=true)
gant.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

gant: org.apache.spark.sql.DataFrame = [Buyer: string, Brand Name: string ... 18 more fields]
root
 |-- Buyer: string (nullable = true)
 |-- Brand Name: string (nullable = true)
 |-- PO Number: long (nullable = true)
 |-- GenCol: string (nullable = true)
 |-- Vendor Code: string (nullable = true)
 |-- Vendor Article No.: long (nullable = true)
 |-- EAN: long (nullable = true)
 |-- Color: string (nullable = true)
 |-- Ven Art No & Color: string (nullable = true)
 |-- Style Code: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Portfolio: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Vertical: string (nullable = true)
 |-- Brick: string (nullable = true)
 |-- Commercial Type: string (nullable = true)
 |-- Season Code: string (nullable = true)
 |-- Option Count: string (nullable = true)
 |-- MRP: integer (nullable = true)



In [6]:
var requiredProducts = gant.select(col("GenCol") as "productid", lower(col("Brand Name")) as "brandname", col("MRP") as "mrp").distinct
requiredProducts.printSchema
println(requiredProducts.count)
requiredProducts.show

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

requiredProducts: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: string, brandname: string ... 1 more field]
root
 |-- productid: string (nullable = true)
 |-- brandname: string (nullable = true)
 |-- mrp: integer (nullable = true)

33
+----------------+---------+-----+
|       productid|brandname|  mrp|
+----------------+---------+-----+
| 465846548_white|     gant|10999|
| 465766062_green|     gant| 7999|
|  465861304_blue|     gant| 7499|
|  465950120_blue|     gant| 9999|
| 465950118_green|     gant| 9999|
|  465876608_blue|     gant| 8499|
| 465769195_green|     gant| 7999|
| 465950109_green|     gant| 8499|
|  465687785_blue|     gant| 7999|
|  465688123_blue|     gant| 8999|
|  465861305_pink|     gant| 7499|
|  465846553_blue|     gant|10999|
|465765226_orange|     gant| 7999|
|  465876615_blue|     gant| 8499|
|  465769197_blue|     gant| 7999|
| 465950116_green|     gant|10999|
| 465688117_green|     gant| 8999|
| 465878750_beige|     gant| 9999|
| 46584

In [16]:
var requiredProductAttrs = requiredProducts.join(productAttrs, Seq(PRODUCTID), "left")
requiredProductAttrs = {
    requiredProductAttrs.withColumn("price", when(col("price").isNotNull, col("price")).otherwise(col("mrp")))
    .withColumn("pricebucket", 
               when(col("price")<=500, lit("1"))
               .when(col("price")<=1000, lit("2"))
               .when(col("price")<=1500, lit("3"))
               .when(col("price")<=2000, lit("4"))
               .when(col("price")<=2500, lit("5"))
               .otherwise(lit(6))
               )
    .withColumn("similargrouplevel", lit("830216013"))
}

requiredProductAttrs.printSchema
println(requiredProductAttrs.count)
requiredProductAttrs.show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

requiredProductAttrs: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 6 more fields]
requiredProductAttrs: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 8 more fields]
root
 |-- productid: string (nullable = true)
 |-- brandname: string (nullable = true)
 |-- mrp: integer (nullable = true)
 |-- colorfamily: string (nullable = true)
 |-- price: double (nullable = true)
 |-- sleeve: string (nullable = true)
 |-- styletype: string (nullable = true)
 |-- pattern: string (nullable = true)
 |-- pricebucket: string (nullable = false)
 |-- similargrouplevel: string (nullable = false)

33
+----------------+---------+-----+-----------+-------+------------------+---------+-------+-----------+-----------------+
|productid       |brandname|mrp  |colorfamily|price  |sleeve            |styletype|pattern|pricebucket|similargrouplevel|
+----------------+---------+-----+-----------+-------+------------------+---------+-------+-----------+----------

In [17]:
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/RequiredProducts", requiredProducts)
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/RequiredProductAttributes", requiredProductAttrs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
var requiredProducts = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/RequiredProducts")
var requiredProductAttrs = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/RequiredProductAttributes")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

requiredProducts: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 1 more field]
requiredProductAttrs: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 8 more fields]


In [19]:
requiredProductAttrs.show(33, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+-----+-----------+-------+------------------+---------+-------+-----------+-----------------+
|productid           |brandname|mrp  |colorfamily|price  |sleeve            |styletype|pattern|pricebucket|similargrouplevel|
+--------------------+---------+-----+-----------+-------+------------------+---------+-------+-----------+-----------------+
|465688111_beige     |gant     |8999 |beige      |4049.55|full-length sleeve|classic  |solid  |6          |830216013        |
|465766062_green     |gant     |7999 |green      |4399.45|full-length sleeve|classic  |stripes|6          |830216013        |
|465846548_white     |gant     |10999|white      |4399.6 |full-length sleeve|classic  |stripes|6          |830216013        |
|465950118_green     |gant     |9999 |green      |3999.6 |full-length sleeve|classic  |solid  |6          |830216013        |
|465765508_yellow    |gant     |7999 |yellow     |3599.55|full-length sleeve|classic  |stripes|6          |830216013  

In [5]:
var productToQuery = {
    requiredProductAttrs.na.fill("#")
    .withColumn("query", concat_ws("_", col("brandname"), col("colorfamily"), col("pattern"), col("pricebucket"), col("sleeve"), col("styletype")))
    .select("productid", "similargrouplevel", "query")
}
productToQuery.show(33, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productToQuery: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 1 more field]
+--------------------+-----------------+------------------------------------------------+
|productid           |similargrouplevel|query                                           |
+--------------------+-----------------+------------------------------------------------+
|465688111_beige     |830216013        |gant_beige_solid_6_full-length sleeve_classic   |
|465766062_green     |830216013        |gant_green_stripes_6_full-length sleeve_classic |
|465846548_white     |830216013        |gant_white_stripes_6_full-length sleeve_classic |
|465950118_green     |830216013        |gant_green_solid_6_full-length sleeve_classic   |
|465765508_yellow    |830216013        |gant_yellow_stripes_6_full-length sleeve_classic|
|465765226_orange    |830216013        |gant_orange_stripes_6_full-length sleeve_classic|
|465840770_green     |830216013        |gant_green_stripes_6_full-length slee

#### Product predictions

In [22]:
var nn_predictions = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/NMonthsAheadExperiment/ProductLevelPredictions")
nn_predictions = {
    nn_predictions.groupBy("productid", "similargrouplevel")
    .pivot("months")
    .agg(sum("predicted"))
}
nn_predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

nn_predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, productid: string ... 2 more fields]
nn_predictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 6 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)



In [25]:
nn_predictions = nn_predictions.join(requiredProducts.select(PRODUCTID).distinct, Seq("productid"))
nn_predictions.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

nn_predictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 6 more fields]
res35: Long = 14


#### Query Avg

In [29]:
var queryAvgPredictions = queryAvg.join(productToQuery, Seq("similargrouplevel", "query"))
queryAvgPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryAvgPredictions: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 7 more fields]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- productid: string (nullable = false)



In [33]:
println(queryAvgPredictions.count)
queryAvgPredictions.show(33, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

32
+-----------------+------------------------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+
|similargrouplevel|query                                           |m1                |m2                |m3                |m4                |m5                |m6                |productid           |
+-----------------+------------------------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+
|830216013        |gant_white_stripes_6_full-length sleeve_classic |1.8039797970226836|1.951472529343196 |2.053013341767447 |1.9422415409769331|2.09832489490509  |1.965381579739707 |465846548_white     |
|830216013        |gant_blue_solid_6_short sleeve_classic          |1.043427750468254 |1.0446764379739761|1.0585337281227112|1.0743151903152466|1.2893744260072708|0.9882813096046448

#### Missing Predictions

In [37]:
var productsHavingPredictions = {
    nn_predictions.select(PRODUCTID).distinct
    .union(queryAvgPredictions.select(PRODUCTID).distinct)
    .distinct
}

var missingProducts = requiredProducts.join(productsHavingPredictions, Seq(PRODUCTID), "leftanti")
println(missingProducts.count)

var missingProductAttrs = missingProducts.drop("mrp", "brandname").join(requiredProductAttrs, Seq(PRODUCTID))
println(missingProductAttrs.count)
missingProductAttrs.show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productsHavingPredictions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: string]
missingProducts: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 1 more field]
1
missingProductAttrs: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 8 more fields]
1
+---------------+---------+-----+-----------+------+------------------+---------+-------+-----------+-----------------+
|productid      |brandname|mrp  |colorfamily|price |sleeve            |styletype|pattern|pricebucket|similargrouplevel|
+---------------+---------+-----+-----------+------+------------------+---------+-------+-----------+-----------------+
|465950113_beige|gant     |10999|beige      |4399.6|full-length sleeve|classic  |stripes|6          |830216013        |
+---------------+---------+-----+-----------+------+------------------+---------+-------+-----------+-----------------+



In [38]:
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/MissingProducts", missingProducts)
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/MissingProductAttrs", missingProductAttrs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
var missingProducts = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/MissingProducts")
var missingProductAttrs = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/MissingProductAttrs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

missingProducts: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 1 more field]
missingProductAttrs: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 8 more fields]


In [44]:
var queryAvgTransformed = {
    queryAvg.withColumn("has_brandname", split(col("query"), "_")(0) =!= "#")
    .withColumn("has_colorfamily", split(col("query"), "_")(1) =!= "#")
    .withColumn("has_pattern", split(col("query"), "_")(2) =!= "#")
    .withColumn("has_pricebucket", split(col("query"), "_")(3) =!= "#")
    .withColumn("has_sleeve", split(col("query"), "_")(4) =!= "#")
    .withColumn("has_styletype", split(col("query"), "_")(5) =!= "#")
    .withColumn("brandname", split(col("query"), "_")(0))
}

var missingProductAttrsTransformed = {
    missingProductAttrs.na.fill("#")
    .withColumn("has_brandname", col("brandname") =!= "#")
    .withColumn("has_colorfamily", col("colorfamily") =!= "#")
    .withColumn("has_pattern", col("pattern") =!= "#")
    .withColumn("has_pricebucket", col("pricebucket") =!= "#")
    .withColumn("has_sleeve", col("sleeve") =!= "#")
    .withColumn("has_styletype", col("styletype") =!= "#")
    .select("productid", "similargrouplevel", "brandname", "has_brandname", "has_colorfamily", "has_pattern", "has_pricebucket", "has_sleeve", "has_styletype")
}

var missingProductsPossibilities = {
    missingProductAttrsTransformed
    .join(queryAvgTransformed, 
          Seq("similargrouplevel", "brandname", "has_brandname", "has_colorfamily", "has_pattern", "has_pricebucket", "has_sleeve", "has_styletype")
         )
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryAvgTransformed: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: int ... 13 more fields]
missingProductAttrsTransformed: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 7 more fields]
missingProductsPossibilities: org.apache.spark.sql.DataFrame = [similargrouplevel: string, brandname: string ... 14 more fields]


In [46]:
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/MissingProductsPossibilities", missingProductsPossibilities)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
var missingProductsPossibilities = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/MissingProductsPossibilities")
missingProductsPossibilities.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

missingProductsPossibilities: org.apache.spark.sql.DataFrame = [similargrouplevel: string, brandname: string ... 14 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- brandname: string (nullable = true)
 |-- has_brandname: boolean (nullable = true)
 |-- has_colorfamily: boolean (nullable = true)
 |-- has_pattern: boolean (nullable = true)
 |-- has_pricebucket: boolean (nullable = true)
 |-- has_sleeve: boolean (nullable = true)
 |-- has_styletype: boolean (nullable = true)
 |-- productid: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)



In [53]:
var missingProductsPredictions = {
    missingProductsPossibilities.groupBy("productid", "similargrouplevel")
    .agg(
        callUDF("percentile", col("m1"), lit(0.5)) as "m1",
        callUDF("percentile", col("m2"), lit(0.5)) as "m2",
        callUDF("percentile", col("m3"), lit(0.5)) as "m3",
        callUDF("percentile", col("m4"), lit(0.5)) as "m4",
        callUDF("percentile", col("m5"), lit(0.5)) as "m5",
        callUDF("percentile", col("m6"), lit(0.5)) as "m6",
        concat_ws(", ", collect_list(col("query"))) as "consideredQueries"
    )
}
missingProductsPredictions.printSchema
println(missingProductsPredictions.count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

missingProductsPredictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 7 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- consideredQueries: string (nullable = false)

1


#### Combine predictions

In [54]:
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/predictions/product", nn_predictions)
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/predictions/queryAvg", queryAvgPredictions)
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/predictions/missing", missingProductsPredictions)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
var productPredictions = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/predictions/product")
var queryAvgPredictions = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/predictions/queryAvg")
var missingPredictions = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/predictions/missing")
productPredictions.printSchema
queryAvgPredictions.printSchema
missingPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productPredictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 6 more fields]
queryAvgPredictions: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 7 more fields]
missingPredictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 7 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)

root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- productid: string (nullable = true)

root
 |-- prod

In [6]:
var actual = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/actual")
actual.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

actual: org.apache.spark.sql.DataFrame = [productid: string, 2023-05: bigint ... 1 more field]
root
 |-- productid: string (nullable = true)
 |-- 2023-05: long (nullable = true)
 |-- 2023-06: long (nullable = true)



In [7]:
productPredictions = {
    productPredictions.join(productToQuery, Seq(PRODUCTID, "similargrouplevel"))
    .withColumn("level", lit("product"))
    .withColumn("consideredQueries", lit("-"))
}
productPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productPredictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 9 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- query: string (nullable = false)
 |-- level: string (nullable = false)
 |-- consideredQueries: string (nullable = false)



In [8]:
queryAvgPredictions = {
    queryAvgPredictions.withColumn("level", lit("queryAvg"))
    .withColumn("consideredQueries", col("query"))
    .join(productPredictions, Seq("similargrouplevel", "productid"), "leftanti")
}
queryAvgPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryAvgPredictions: org.apache.spark.sql.DataFrame = [similargrouplevel: int, productid: string ... 9 more fields]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- productid: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- level: string (nullable = false)
 |-- consideredQueries: string (nullable = true)



In [9]:
missingPredictions = {
    missingPredictions.join(productToQuery, Seq(PRODUCTID, "similargrouplevel"))
    .withColumn("level", lit("missing"))
}
missingPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

missingPredictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 9 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- consideredQueries: string (nullable = true)
 |-- query: string (nullable = false)
 |-- level: string (nullable = false)



In [10]:
var finalPredictions = {
    productPredictions.union(queryAvgPredictions.select(productPredictions.columns.map(col): _*))
    .union(missingPredictions.select(productPredictions.columns.map(col): _*))
    .join(actual, Seq("productid"), "left")
    .na.fill(0)
    .withColumn("m1", round(col("m1")).cast(IntegerType))
    .withColumnRenamed("m1", "May-Prediction")
    .withColumn("m2", round(col("m2")).cast(IntegerType))
    .withColumnRenamed("m2", "June-Prediction")
    .withColumn("m3", round(col("m3")).cast(IntegerType))
    .withColumnRenamed("m3", "July-Prediction")
    .withColumn("m4", round(col("m4")).cast(IntegerType))
    .withColumnRenamed("m4", "August-Prediction")
    .withColumn("m5", round(col("m5")).cast(IntegerType))
    .withColumnRenamed("m5", "September-Prediction")
    .withColumn("m6", round(col("m6")).cast(IntegerType))
    .withColumnRenamed("m6", "October-Prediction")
    .withColumnRenamed("2023-05", "May-Actual")
    .withColumnRenamed("2023-06", "June-Actual")
    .select(
        ("productid,similargrouplevel,May-Prediction,June-Prediction,July-Prediction,August-Prediction,September-Prediction,October-Prediction,query"
        + ",level,consideredQueries,May-Actual,June-Actual").split(",").map(col): _*
    )
}
finalPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

finalPredictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 11 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- May-Prediction: integer (nullable = true)
 |-- June-Prediction: integer (nullable = true)
 |-- July-Prediction: integer (nullable = true)
 |-- August-Prediction: integer (nullable = true)
 |-- September-Prediction: integer (nullable = true)
 |-- October-Prediction: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- level: string (nullable = false)
 |-- consideredQueries: string (nullable = true)
 |-- May-Actual: long (nullable = false)
 |-- June-Actual: long (nullable = false)



In [11]:
DFToCSV.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/gant/RequiredPredictions", finalPredictions.coalesce(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Aarke

In [5]:
var aarke = CSVToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/M&S Gant Aarke - Aarke.csv", inferSchema=true)
aarke.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

aarke: org.apache.spark.sql.DataFrame = [Buyer: string, Brand Name: string ... 16 more fields]
root
 |-- Buyer: string (nullable = true)
 |-- Brand Name: string (nullable = true)
 |-- GenCol: string (nullable = true)
 |-- EAN: long (nullable = true)
 |-- Style code: string (nullable = true)
 |-- Vendor Article No.: long (nullable = true)
 |-- Color: string (nullable = true)
 |-- Ven Art No & Color: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Portfolio: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Vertical: string (nullable = true)
 |-- Brick: string (nullable = true)
 |-- Commercial Type: string (nullable = true)
 |-- Season Code: string (nullable = true)
 |-- Option Count: integer (nullable = true)
 |-- MRP: integer (nullable = true)



In [6]:
var requiredProducts = {
    aarke.select(col("GenCol") as "productid", lower(col("Brand Name")) as "brandname", col("MRP") as "mrp")
    .distinct
    .withColumn("brandname", lit("aarke-ritu-kumar"))
}
requiredProducts.printSchema
println(requiredProducts.count)
requiredProducts.show

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

requiredProducts: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 1 more field]
root
 |-- productid: string (nullable = true)
 |-- brandname: string (nullable = false)
 |-- mrp: integer (nullable = true)

20
+--------------------+----------------+----+
|           productid|       brandname| mrp|
+--------------------+----------------+----+
|      410334557_blue|aarke-ritu-kumar|2400|
|    410334456_purple|aarke-ritu-kumar|2700|
|     410334361_green|aarke-ritu-kumar|3400|
|      410334548_pink|aarke-ritu-kumar|3500|
|      410334391_blue|aarke-ritu-kumar|3200|
|      410334354_blue|aarke-ritu-kumar|3800|
|      410334280_blue|aarke-ritu-kumar|2800|
|    410331534_purple|aarke-ritu-kumar|3500|
|      410334556_pink|aarke-ritu-kumar|2500|
|     410334382_peach|aarke-ritu-kumar|2500|
|      410334347_blue|aarke-ritu-kumar|1500|
|    410334372_yellow|aarke-ritu-kumar|2900|
|      410318152_ecru|aarke-ritu-kumar|2200|
|      410334350_navy|aarke-ritu-kumar|2500|


In [7]:
var requiredProductAttrs = requiredProducts.join(productAttrs, Seq(PRODUCTID), "left")
requiredProductAttrs = {
    requiredProductAttrs.withColumn("price", when(col("price").isNotNull, col("price")).otherwise(col("mrp")))
    .withColumn("pricebucket", 
               when(col("price")<=500, lit("1"))
               .when(col("price")<=1000, lit("2"))
               .when(col("price")<=1500, lit("3"))
               .when(col("price")<=2000, lit("4"))
               .when(col("price")<=2500, lit("5"))
               .otherwise(lit(6))
               )
    .withColumn("similargrouplevel", lit("830303011"))
}

requiredProductAttrs.printSchema
println(requiredProductAttrs.count)
requiredProductAttrs.show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

requiredProductAttrs: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 6 more fields]
requiredProductAttrs: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 8 more fields]
root
 |-- productid: string (nullable = true)
 |-- brandname: string (nullable = false)
 |-- mrp: integer (nullable = true)
 |-- colorfamily: string (nullable = true)
 |-- price: double (nullable = true)
 |-- sleeve: string (nullable = true)
 |-- styletype: string (nullable = true)
 |-- pattern: string (nullable = true)
 |-- pricebucket: string (nullable = false)
 |-- similargrouplevel: string (nullable = false)

20
+--------------------+----------------+----+-----------+------+------------------+---------+----------+-----------+-----------------+
|productid           |brandname       |mrp |colorfamily|price |sleeve            |styletype|pattern   |pricebucket|similargrouplevel|
+--------------------+----------------+----+-----------+------+------------------+------

In [8]:
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/RequiredProducts", requiredProducts)
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/RequiredProductAttributes", requiredProductAttrs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
var requiredProducts = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/RequiredProducts")
var requiredProductAttrs = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/RequiredProductAttributes")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

requiredProducts: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 1 more field]
requiredProductAttrs: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 8 more fields]


In [10]:
var productToQuery = {
    requiredProductAttrs.na.fill("#")
    .withColumn("query", concat_ws("_", col("brandname"), col("colorfamily"), col("pattern"), col("pricebucket"), col("sleeve"), col("styletype")))
    .select("productid", "similargrouplevel", "query")
}
productToQuery.show(33, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productToQuery: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 1 more field]
+--------------------+-----------------+---------------------------------------------------------+
|productid           |similargrouplevel|query                                                    |
+--------------------+-----------------+---------------------------------------------------------+
|410334279_red       |830303011        |aarke-ritu-kumar_red_floral_6_3-4th sleeve_straight      |
|410334350_beige     |830303011        |aarke-ritu-kumar_beige_floral_4_full-length sleeve_empire|
|410331534_purple    |830303011        |aarke-ritu-kumar_purple_embroidery_5_short sleeve_a-line |
|410334548_pink      |830303011        |aarke-ritu-kumar_pink_floral_6_full-length sleeve_a-line |
|410334350_navy      |830303011        |aarke-ritu-kumar_navy_floral_4_full-length sleeve_empire |
|410334382_powderblue|830303011        |aarke-ritu-kumar_blue_indian_5_3-4th sleeve_a-line     

#### Product predictions

In [11]:
var nn_predictions = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/NMonthsAheadExperiment/ProductLevelPredictions")
nn_predictions = {
    nn_predictions.groupBy("productid", "similargrouplevel")
    .pivot("months")
    .agg(sum("predicted"))
}
nn_predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

nn_predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, productid: string ... 2 more fields]
nn_predictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 6 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)



In [12]:
nn_predictions = nn_predictions.join(requiredProducts.select(PRODUCTID).distinct, Seq("productid"))
nn_predictions.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

nn_predictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 6 more fields]
res19: Long = 6


In [13]:
nn_predictions.join(productToQuery, Seq(PRODUCTID, "similargrouplevel")).select("productid", "similargrouplevel", "query").show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----------------+--------------------------------------------------------+
|productid       |similargrouplevel|query                                                   |
+----------------+-----------------+--------------------------------------------------------+
|410331534_purple|830303011        |aarke-ritu-kumar_purple_embroidery_5_short sleeve_a-line|
|410334456_purple|830303011        |aarke-ritu-kumar_purple_paisley_4_#_straight            |
|410334361_green |830303011        |aarke-ritu-kumar_green_floral_6_3-4th sleeve_a-line     |
|410334279_red   |830303011        |aarke-ritu-kumar_red_floral_6_3-4th sleeve_straight     |
|410334372_yellow|830303011        |aarke-ritu-kumar_yellow_floral_5_3-4th sleeve_a-line    |
|410318152_ecru  |830303011        |aarke-ritu-kumar_multi_paisley_2_3-4th sleeve_a-line    |
+----------------+-----------------+--------------------------------------------------------+



#### Query avg predictions

In [14]:
var queryAvgPredictions = queryAvg.join(productToQuery, Seq("similargrouplevel", "query"))
queryAvgPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryAvgPredictions: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 7 more fields]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- productid: string (nullable = false)



In [15]:
println(queryAvgPredictions.count)
queryAvgPredictions.show(33, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4
+-----------------+----------------------------------------------------+------------------+------------------+------------------+------------------+-----------------+------------------+----------------+
|similargrouplevel|query                                               |m1                |m2                |m3                |m4                |m5               |m6                |productid       |
+-----------------+----------------------------------------------------+------------------+------------------+------------------+------------------+-----------------+------------------+----------------+
|830303011        |aarke-ritu-kumar_yellow_floral_5_3-4th sleeve_a-line|6.228254079818726 |6.650531053543091 |7.394933223724365 |7.087536811828613 |8.609011888504028|7.469300985336304 |410334372_yellow|
|830303011        |aarke-ritu-kumar_#_#_6_#_#                          |2.4344528913497925|2.5854403972625732|2.7305211424827576|2.7982500791549683|2.990735173225403|3.0211694836616516|4

#### Missing Predictions

In [16]:
var productsHavingPredictions = {
    nn_predictions.select(PRODUCTID).distinct
    .union(queryAvgPredictions.select(PRODUCTID).distinct)
    .distinct
}

var missingProducts = requiredProducts.join(productsHavingPredictions, Seq(PRODUCTID), "leftanti")
println(missingProducts.count)

var missingProductAttrs = missingProducts.drop("mrp", "brandname").join(requiredProductAttrs, Seq(PRODUCTID))
println(missingProductAttrs.count)
missingProductAttrs.show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productsHavingPredictions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: string]
missingProducts: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 1 more field]
13
missingProductAttrs: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 8 more fields]
13
+--------------------+----------------+----+-----------+------+------------------+---------+---------+-----------+-----------------+
|productid           |brandname       |mrp |colorfamily|price |sleeve            |styletype|pattern  |pricebucket|similargrouplevel|
+--------------------+----------------+----+-----------+------+------------------+---------+---------+-----------+-----------------+
|410334350_beige     |aarke-ritu-kumar|2500|beige      |1925.0|full-length sleeve|empire   |floral   |4          |830303011        |
|410334548_pink      |aarke-ritu-kumar|3500|pink       |3500.0|full-length sleeve|a-line   |floral   |6          |830303011        |
|4103343

In [17]:
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/MissingProducts", missingProducts)
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/MissingProductAttrs", missingProductAttrs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
var missingProducts = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/MissingProducts")
var missingProductAttrs = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/MissingProductAttrs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

missingProducts: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 1 more field]
missingProductAttrs: org.apache.spark.sql.DataFrame = [productid: string, brandname: string ... 8 more fields]


In [20]:
var queryAvgTransformed = {
    queryAvg.withColumn("has_brandname", split(col("query"), "_")(0) =!= "#")
    .withColumn("has_colorfamily", split(col("query"), "_")(1) =!= "#")
    .withColumn("has_pattern", split(col("query"), "_")(2) =!= "#")
    .withColumn("has_pricebucket", split(col("query"), "_")(3) =!= "#")
    .withColumn("has_sleeve", split(col("query"), "_")(4) =!= "#")
    .withColumn("has_styletype", split(col("query"), "_")(5) =!= "#")
    .withColumn("brandname", split(col("query"), "_")(0))
}

var missingProductAttrsTransformed = {
    missingProductAttrs.na.fill("#")
    .withColumn("has_brandname", col("brandname") =!= "#")
    .withColumn("has_colorfamily", col("colorfamily") =!= "#")
    .withColumn("has_pattern", col("pattern") =!= "#")
    .withColumn("has_pricebucket", col("pricebucket") =!= "#")
    .withColumn("has_sleeve", col("sleeve") =!= "#")
    .withColumn("has_styletype", col("styletype") =!= "#")
    .select("productid", "similargrouplevel", "brandname", "has_brandname", "has_colorfamily", "has_pattern", "has_pricebucket", "has_sleeve", "has_styletype")
}

var missingProductsPossibilities = {
    missingProductAttrsTransformed
    .join(queryAvgTransformed, 
          Seq("similargrouplevel", "brandname", "has_brandname", "has_colorfamily", "has_pattern", "has_pricebucket", "has_sleeve", "has_styletype")
         )
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryAvgTransformed: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: int ... 13 more fields]
missingProductAttrsTransformed: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 7 more fields]
missingProductsPossibilities: org.apache.spark.sql.DataFrame = [similargrouplevel: string, brandname: string ... 14 more fields]


In [21]:
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/MissingProductsPossibilities", missingProductsPossibilities)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
var missingProductsPossibilities = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/MissingProductsPossibilities")
missingProductsPossibilities.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

missingProductsPossibilities: org.apache.spark.sql.DataFrame = [similargrouplevel: string, brandname: string ... 14 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- brandname: string (nullable = true)
 |-- has_brandname: boolean (nullable = true)
 |-- has_colorfamily: boolean (nullable = true)
 |-- has_pattern: boolean (nullable = true)
 |-- has_pricebucket: boolean (nullable = true)
 |-- has_sleeve: boolean (nullable = true)
 |-- has_styletype: boolean (nullable = true)
 |-- productid: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)



In [23]:
var missingProductsPredictions = {
    missingProductsPossibilities.groupBy("productid", "similargrouplevel")
    .agg(
        callUDF("percentile", col("m1"), lit(0.5)) as "m1",
        callUDF("percentile", col("m2"), lit(0.5)) as "m2",
        callUDF("percentile", col("m3"), lit(0.5)) as "m3",
        callUDF("percentile", col("m4"), lit(0.5)) as "m4",
        callUDF("percentile", col("m5"), lit(0.5)) as "m5",
        callUDF("percentile", col("m6"), lit(0.5)) as "m6",
        concat_ws(", ", collect_list(col("query"))) as "consideredQueries"
    )
}
missingProductsPredictions.printSchema
println(missingProductsPredictions.count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

missingProductsPredictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 7 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- consideredQueries: string (nullable = false)

13


#### Combined Predictions

In [24]:
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/predictions/product", nn_predictions)
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/predictions/queryAvg", queryAvgPredictions)
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/predictions/missing", missingProductsPredictions)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
var productPredictions = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/predictions/product")
var queryAvgPredictions = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/predictions/queryAvg")
var missingPredictions = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/predictions/missing")
productPredictions.printSchema
queryAvgPredictions.printSchema
missingPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productPredictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 6 more fields]
queryAvgPredictions: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 7 more fields]
missingPredictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 7 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)

root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- productid: string (nullable = true)

root
 |-- prod

In [26]:
var actual = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/actual")
actual.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

actual: org.apache.spark.sql.DataFrame = [productid: string, 2023-05: bigint ... 1 more field]
root
 |-- productid: string (nullable = true)
 |-- 2023-05: long (nullable = true)
 |-- 2023-06: long (nullable = true)



In [27]:
productPredictions = {
    productPredictions.join(productToQuery, Seq(PRODUCTID, "similargrouplevel"))
    .withColumn("level", lit("product"))
    .withColumn("consideredQueries", lit("-"))
}
productPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productPredictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 9 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- query: string (nullable = false)
 |-- level: string (nullable = false)
 |-- consideredQueries: string (nullable = false)



In [28]:
queryAvgPredictions = {
    queryAvgPredictions.withColumn("level", lit("queryAvg"))
    .withColumn("consideredQueries", col("query"))
    .join(productPredictions, Seq("similargrouplevel", "productid"), "leftanti")
}
queryAvgPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryAvgPredictions: org.apache.spark.sql.DataFrame = [similargrouplevel: int, productid: string ... 9 more fields]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- productid: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- level: string (nullable = false)
 |-- consideredQueries: string (nullable = true)



In [29]:
missingPredictions = {
    missingPredictions.join(productToQuery, Seq(PRODUCTID, "similargrouplevel"))
    .withColumn("level", lit("missing"))
}
missingPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

missingPredictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 9 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- consideredQueries: string (nullable = true)
 |-- query: string (nullable = false)
 |-- level: string (nullable = false)



In [30]:
var finalPredictions = {
    productPredictions.union(queryAvgPredictions.select(productPredictions.columns.map(col): _*))
    .union(missingPredictions.select(productPredictions.columns.map(col): _*))
    .join(actual, Seq("productid"), "left")
    .na.fill(0)
    .withColumn("m1", round(col("m1")).cast(IntegerType))
    .withColumnRenamed("m1", "May-Prediction")
    .withColumn("m2", round(col("m2")).cast(IntegerType))
    .withColumnRenamed("m2", "June-Prediction")
    .withColumn("m3", round(col("m3")).cast(IntegerType))
    .withColumnRenamed("m3", "July-Prediction")
    .withColumn("m4", round(col("m4")).cast(IntegerType))
    .withColumnRenamed("m4", "August-Prediction")
    .withColumn("m5", round(col("m5")).cast(IntegerType))
    .withColumnRenamed("m5", "September-Prediction")
    .withColumn("m6", round(col("m6")).cast(IntegerType))
    .withColumnRenamed("m6", "October-Prediction")
    .withColumnRenamed("2023-05", "May-Actual")
    .withColumnRenamed("2023-06", "June-Actual")
    .select(
        ("productid,similargrouplevel,May-Prediction,June-Prediction,July-Prediction,August-Prediction,September-Prediction,October-Prediction,query"
        + ",level,consideredQueries,May-Actual,June-Actual").split(",").map(col): _*
    )
}
finalPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

finalPredictions: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 11 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- May-Prediction: integer (nullable = true)
 |-- June-Prediction: integer (nullable = true)
 |-- July-Prediction: integer (nullable = true)
 |-- August-Prediction: integer (nullable = true)
 |-- September-Prediction: integer (nullable = true)
 |-- October-Prediction: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- level: string (nullable = false)
 |-- consideredQueries: string (nullable = true)
 |-- May-Actual: long (nullable = false)
 |-- June-Actual: long (nullable = false)



In [31]:
DFToCSV.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/ajio/aarke-ritu-kumar/RequiredPredictions", finalPredictions.coalesce(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…